<a href="https://colab.research.google.com/github/lorenzo-crippa/3M_NLP_ESS_2022/blob/main/Tutorial_Three_(R)_Introduction_to_Word_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Word Embeddings

## Douglas Rice

In this notebook, we'll estimate our first word embedding model, then go through a series of analyses of the estimated embeddings. After completing this notebook, you should be familar with:


1. Preparing a corpus for estimating word embeddings
2. Estimating a (static) word embedding model
3. Analyzing output of (static) word embedding model





# GloVe

We'll be using the [`text2vec`](http://text2vec.org/index.html) package. `text2vec` was one of the first implementations of  word embeddings functionality in R, and is designed to run *fast*, relatively speaking. Still, it's important to remember that our computational complexity is amping up here, so don't expect immediate results. 

`text2vec` implements the "Global Vectors" (or GloVe) approach for estimating embeddings. Stanford University's [Global Vectors for Word Representation (GloVe)](https://nlp.stanford.edu/projects/glove/) is an approach to estimating a distributional representation of a word. GloVe is based, essentially, on factorizing a huge term co-occurrence matrix. 

The distributional representation of words means that each term is represented as a distribution over some number of dimensions (say, 3 dimensions, where the values are 0.6, 0.3, and 0.1). This stands in stark contrast to the work we've done to this point, which has effectively encoded each word as being effectively just present (1) or not (0). 

Perhaps unsurprisingly, the distributional representation better captures semantic meaning than the one-hot encoding. This opens up a world of possibilities for us as researchers. Indeed, this has been a major leap forward for research in Text-as-Data. 

As an example, we can see how similar one word is to other words by measuring the distance between their distributions. Even more interestingly, we can capture really specific phenomena from text with some simple arithmetic based on word distributions. Consider the following canonical example:

- <h2> king - man + woman = queen </h2>

Ponder this equation for a moment. From the vector representation of  **king**, we subtract the vector representation of **man**. Then, we add the vector representation of **woman**. The end result of that should be a vector that is very similar to the vector representation of  **queen**. 

In what follows, we'll work through some examples to see how well this works. I want to caution, though, that the models we are training here are probably too small for us to have too much confidence in the trained models. Nevertheless, you'll see that even with this small set we'll recover really interesting dynamics.



## Front-end Matters

First, let's install the `text2vec` package:

In [1]:
# Installs text2vec package (might take a while)
install.packages('text2vec')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘MatrixExtra’, ‘float’, ‘RhpcBLASctl’, ‘RcppArmadillo’, ‘Rcpp’, ‘rsparse’, ‘mlapi’, ‘lgr’




And load the library:

In [6]:
library(text2vec)

## PoKi Dataset

We'll be using [PoKi](https://github.com/whipson/PoKi-Poems-by-Kids), a corpus of poems written by children and teenagers from grades 1 to 12.

One thing to flag right off the bat is the really interesting dynamics related to *who* is writing these posts. We need to keep in mind that the children writing these texts are going to use less formal writing and more imaginative stories. Given this, we'll focus on analogies that are more appropriate for this context; here, we'll aim to create word embeddings that can recreate these two equations:

- <h2> cat - meow + bark = dog </h2>

- <h2> mom - girl + boy = dad </h2>

By the end, we should hopefully be able to recreate these by creating and fitting our GloVe models. But first, let's perform the necessary pre-processing steps before creating our embedding models. 

Let's download and read in the data:



In [7]:
# Creates file
temp <- tempfile()

# Downloads and unzips file to a text8_file variable if it does not exist
download.file("https://raw.githubusercontent.com/whipson/PoKi-Poems-by-Kids/master/poki.csv", temp)

In [8]:
# Reads in downloaded file
poem <- read.csv(temp)

# First ten rows
head(poem, 10)

,id,title,author,grade,text,char
,<int>,<chr>,<chr>,<int>,<chr>,<int>
1,104987,I Love The Zoo,,1,"roses are red, violets are blue. i love the zoo. do you?",62
2,67185,The scary forest.,,1,the forest is really haunted. i believe it to be so. but then we are going camping.,87
3,103555,A Hike At School,1st grade-wh,1,"i took a hike at school today and this is what i saw bouncing balls girls chatting against the walls kids climbing on monkey bars i even saw some teachers' cars the wind was blowing my hair in my face i saw a mud puddle, but just a trace all of these things i noticed just now on my little hike.",324
4,112483,Computer,a,1,"you can do what you want you can play a game you can do many things, you can read and write",106
5,74516,Angel,aab,1,angel oh angle you spin like a top angel oh angel you will never stop can't you feel the air as it blows through your hair angel oh angel itisto bad your a mop!,164
6,114693,Nature Nature and Nature,aadhya,1,"look at the sun, what a beautiful day. under the trees, we can run and play. beauty of nature, we love to see, from tiny insect to exotic tree. it is a place to sit and think, nature and human share the deepest link. nature has ocean, which is in motion. nature has tree, nature has river. if we destroy the nature we would never be free. our nature keeps us alive, we must protect it, for society to thrive. we spoil the nature, we spoil the future. go along with nature, for your better future.",491
7,46453,Jack,aaliyah,1,"dog playful, energetic running, jumping, tackling my is my friend jack",74
8,57397,When I awoke one morning,aanna,1,"when i awoke one morning, a dog was on my head. i asked , ''what are you doing there?' it looked at me and said ''woof!'' ''wouldn't you like to be outside playing?''said the man ''i'm staying here and playing here. '' said the dog he played all night and day. he came inside his new house and played inside a wet wet day.",325
9,77201,My Blue Berries and My Cherries,aarathi,1,i went to my blue berry tree they were no blue berries found i went to another tree to get some more free but found none but cherries round.,143


In [9]:
# Checks dimensions
dim(poem) # 61,508 poems

[1] 61508     6

We want the poems themselves, so we'll use the column 'text' for tokenization.

## Tokenization and Vectorization

We start with `text2vec` by creating a tokenized iterator and vectorized vocabulary first. This time, there's no need to lowercase our words since the downloaded dataset is already lowercased.

Let's tokenize the data:

In [10]:
# Tokenization
tokens <- word_tokenizer(poem$text)

# First five rows tokenized
head(tokens, 5)

[[1]]
 [1] "roses"   "are"     "red"     "violets" "are"     "blue"    "i"      
 [8] "love"    "the"     "zoo"     "do"      "you"    

[[2]]
 [1] "the"     "forest"  "is"      "really"  "haunted" "i"       "believe"
 [8] "it"      "to"      "be"      "so"      "but"     "then"    "we"     
[15] "are"     "going"   "camping"

[[3]]
 [1] "i"        "took"     "a"        "hike"     "at"       "school"  
 [7] "today"    "and"      "this"     "is"       "what"     "i"       
[13] "saw"      "bouncing" "balls"    "girls"    "chatting" "against" 
[19] "the"      "walls"    "kids"     "climbing" "on"       "monkey"  
[25] "bars"     "i"        "even"     "saw"      "some"     "teachers"
[31] "cars"     "the"      "wind"     "was"      "blowing"  "my"      
[37] "hair"     "in"       "my"       "face"     "i"        "saw"     
[43] "a"        "mud"      "puddle"   "but"      "just"     "a"       
[49] "trace"    "all"      "of"       "these"    "things"   "i"       
[55] "noticed"  "just"     "now"      "on"       "my"       "little"  
[61] "hike"    

[[4]]
 [1] "you"    "can"    "do"     "what"   "you"    "want"   "you"    "can"   
 [9] "play"   "a"      "game"   "you"    "can"    "do"     "many"   "things"
[17] "you"    "can"    "read"   "and"    "write" 

[[5]]
 [1] "angel"   "oh"      "angle"   "you"     "spin"    "like"    "a"      
 [8] "top"     "angel"   "oh"      "angel"   "you"     "will"    "never"  
[15] "stop"    "can't"   "you"     "feel"    "the"     "air"     "as"     
[22] "it"      "blows"   "through" "your"    "hair"    "angel"   "oh"     
[29] "angel"   "itisto"  "bad"     "your"    "a"       "mop"

Create an iterator object:

In [12]:
# Create iterator object 
it <- itoken(tokens, progressbar = FALSE)
it

<itoken>
  Inherits from: <CallbackIterator>
  Public:
    callback: function (x) 
    clone: function (deep = FALSE) 
    initialize: function (x, callback = identity) 
    is_complete: active binding
    length: active binding
    move_cursor: function () 
    nextElem: function () 
    x: GenericIterator, iterator, R6

Build the vocabulary:

In [13]:
#?create_vocabulary

In [16]:
# Build vocabulary
vocab <- create_vocabulary(it)

# Vocabulary
vocab # we have term, count for the terms overall, and the number of documents it occurs in

term,term_count,doc_count
<chr>,<int>,<int>
0000,1,1
0000000,1,1
0000001,1,1
00a:m,1,1
00he,1,1
00o'clock,1,1
00p,1,1
02,1,1
04,1,1


In [17]:
# Check dimensions
dim(vocab) # the vocabulary has 56,474 types. We have word count and document counts

[1] 56474     3

And prune and vectorize it. We'll keep the terms that occur at least 5 times.

In [21]:
# Prune vocabulary
vocab <- prune_vocabulary(vocab, term_count_min = 5)

# Check dimensions
dim(vocab) # many many fewer words: 14k, about

# Vectorize
vectorizer <- vocab_vectorizer(vocab) # vectorizer of the vocabulary that we care about

[1] 14267     3

As we can see, pruning our vocabulary deleted over 40 thousand words. I want to reiterate that this is a *very small* corpus from the perspective of traditional word embedding models. When we are working with word representations trained with these smaller corpora, we should be really cautious in our approach. 

Moving on, we can create out term-co-occurence matrix (TCM). We can achieve different results by experimenting with the `skip_grams_window` and other parameters. The definition of whether two words occur together is arbitrary, so we definitely want to play around with the parameters to see the different results.

In [24]:
# use window of 5 for context words
tcm <- create_tcm(it, vectorizer, skip_grams_window = 5L) 
# so we need: the iterator we created earlier (notice: unpruned at this point), 
# the vectorizer (defining the *pruned* vocabulary we care about), and the
# size of the skip-grams window we want (in this case, 5 words)

## Creating and fitting the GloVe model

Now we have a TCM matrix and can factorize it via the GloVe algorithm. We'll use the method `$new` to `GlobalVectors` to create our GloVe model. 

[Here](https://www.rdocumentation.org/packages/text2vec/versions/0.5.0/topics/GlobalVectors) is documentation for related functions and methods.

In [25]:
# Creating new GloVe model
glove <- GlobalVectors$new(rank = 50, x_max = 10) # rank = 50 determines the number of dimension of the final model (number of weights per word)
# notice that this function works in an object-oriented fashion. We take the attribute new() from GlobalVectors

# Checking GloVe methods
glove

<GloVe>
  Public:
    bias_i: NULL
    bias_j: NULL
    clone: function (deep = FALSE) 
    components: NULL
    fit_transform: function (x, n_iter = 10L, convergence_tol = -1, n_threads = getOption("rsparse_omp_threads", 
    get_history: function () 
    initialize: function (rank, x_max, learning_rate = 0.15, alpha = 0.75, lambda = 0, 
    shuffle: FALSE
  Private:
    alpha: 0.75
    b_i: NULL
    b_j: NULL
    cost_history: 
    fitted: FALSE
    glove_fitter: NULL
    initial: NULL
    lambda: 0
    learning_rate: 0.15
    rank: 50
    w_i: NULL
    w_j: NULL
    x_max: 10

Note that you'll only be able to access the public methods.

We can fit our model using `$fit_transform` to our `glove` variable. This may take several minutes to fit! 

In [26]:
# Fitting model
wv_main <- glove$fit_transform(tcm, n_iter = 10, convergence_tol = 0.01, n_threads = 8)
# advices on default options:
# n_iter:
# convergence_tol:
# n_threads: 

INFO  [16:20:26.670] epoch 1, loss 0.1998 
INFO  [16:20:29.406] epoch 2, loss 0.1303 
INFO  [16:20:32.127] epoch 3, loss 0.1127 
INFO  [16:20:34.836] epoch 4, loss 0.1018 
INFO  [16:20:37.473] epoch 5, loss 0.0944 
INFO  [16:20:40.122] epoch 6, loss 0.0889 
INFO  [16:20:42.753] epoch 7, loss 0.0848 
INFO  [16:20:45.371] epoch 8, loss 0.0816 
INFO  [16:20:48.145] epoch 9, loss 0.0790 
INFO  [16:20:50.908] epoch 10, loss 0.0768 


In [27]:
# Checking dimensions
dim(wv_main)

[1] 14267    50

Note that model learns *two* sets of word vectors - **target** and **context**. We can think of our word of interest as the target in this environment, and all the other words as the context inside the window. For both, word vectors are learned. 

In [28]:
wv_context <- glove$components
dim(wv_context)

[1]    50 14267

While both of word-vectors matrices can be used as result, the creators recommends to average or take a sum of main and context vector:

In [29]:
word_vectors <- wv_main + t(wv_context)

Here's a preview of the word vector matrix:

In [30]:
word_vectors

1837,-0.63303339,0.30040185,-0.16941897,0.19313955,0.33992052,0.10447346,-0.61349224,-0.51546688,1.143208193,0.50096603,⋯,-0.207672794,-0.68709990,-0.16135047,-0.27323195,0.074283333,-0.440089791,-0.294238013,0.10534551,0.6299827557,0.528825835
1841,-0.64363169,0.39464305,-0.51663717,-0.26036606,0.17839496,0.28172289,0.35276825,0.04594110,-0.085550141,0.60269109,⋯,-0.078028838,-0.84906721,-0.74034014,0.22099173,-0.005637184,-0.196269850,-0.490606502,-0.14352095,-0.5746922442,0.484796474
1881,-0.16064160,0.63842416,-0.15512472,0.21920492,-0.40337032,0.08934645,0.05465616,-0.68041639,0.486641403,0.33557877,⋯,0.523296084,0.10808258,-0.14250742,-0.30209468,-0.347134508,-0.131963484,0.199057038,-0.46297884,0.3299895984,-0.023089008
2005,0.30330476,0.46340260,1.07200098,0.64906417,-0.09673991,0.24447156,-0.22427952,0.05807113,0.176181114,0.77190048,⋯,0.217931912,-0.23449325,-0.54430110,0.74694629,0.378682670,-0.414246425,-0.251819016,0.09975650,-0.3260630406,0.313982341
36,-1.04820731,0.18680421,-0.36206242,-0.31433877,-0.16047504,0.02559040,-0.52615126,0.30309286,0.594394198,0.15275491,⋯,-0.432263762,-0.94327215,-0.86728761,0.03697629,0.023473632,0.250202793,-0.225504402,0.56372624,0.3423216271,0.170207423
38,0.23950902,0.66162068,0.40454963,-0.12451774,-0.27237319,-0.25242899,-0.32643207,0.77775335,0.008975544,0.54310462,⋯,0.002088803,-0.40748182,0.14296547,-0.57684190,-0.405189757,-0.280860809,-0.102424417,-0.13971630,0.1082684066,0.846399960
39,-0.45500920,0.37761543,0.12899650,0.34855021,0.95821330,0.22457393,0.27838264,0.27735947,0.175953891,-0.13082525,⋯,-0.223028636,0.25634268,-0.61034492,0.08320432,-0.007539595,0.823494643,-0.504143283,-0.23752902,0.5533741496,0.046986448
52,-0.05806557,0.05131635,1.13282643,0.22139621,0.68497688,0.09284366,-0.25984258,-1.01686067,-0.238540322,-0.14508061,⋯,-0.016681551,-0.84191835,0.13537092,0.93538656,0.331386129,-0.284657078,0.037629748,0.77218520,0.0445297803,-0.179013608
5â,-0.33385473,0.15762580,0.69672298,0.06397768,-0.38596070,0.44789943,0.46258904,0.51563506,0.232720042,0.36631229,⋯,0.770611898,-0.51251799,-0.14484181,0.03746398,-0.351322133,-0.531175056,-1.027938904,-0.24506040,-0.2788825331,0.262615112
600,-0.33060412,0.50694422,0.08683890,-0.07372704,-0.19431043,-0.51469502,-0.15987011,-0.83610260,-0.196220839,0.08482536,⋯,0.066924373,-0.81379241,-0.69483290,-0.94866798,-0.504177778,-0.240210791,0.055614033,0.22476259,-0.3597520838,-0.409118558
abcdefg,-0.40157881,0.01796207,0.64565495,-0.70716491,1.33364934,-0.31949992,-0.26349394,-0.31911708,-0.268414973,0.57926403,⋯,-0.380683508,-0.56358408,-0.60721345,0.37689570,-0.294437593,-0.349698620,0.312965267,-0.34318367,-0.2756320172,0.390639932


## Cosine Similarity

Now we can begin to play. Similarly to standard correlation, we can look at comparing two vectors using **cosine similarity**. Let's see what is similar with 'school':

In [33]:
# Word vector for school
school <- word_vectors["school", , drop = FALSE]

# Cosine similarity
school_cos_sim <- sim2(x = word_vectors, y = school, method = "cosine", norm = "l2")

# Top ten words relating to school
head(round(sort(school_cos_sim[,1], decreasing = TRUE), digits = 3), 10)

school  today    day   work   time     we   pool    fun   cool    end 
 1.000  0.690  0.670  0.664  0.659  0.643  0.636  0.634  0.627  0.625

Obviously, school is the most similar to school. Based on the poems that the children wrote, we can also see words like 'work', 'fun' and 'home' as most similar to 'school'.

## Pet example

Let's try our pet example:

In [35]:
# cat - meow + bark should equal dog
dog <- word_vectors["cat", , drop = FALSE] -
  word_vectors["meow", , drop = FALSE] +
  word_vectors["bark", , drop = FALSE]
  
# Calculates pairwise similarities between the rows of two matrices
dog_cos_sim <- sim2(x = word_vectors, y = dog, method = "cosine", norm = "l2")

# Top five predictions
head(round(sort(dog_cos_sim[,1], decreasing = TRUE), digits = 4), 5)

cat    dog    fat    rat  likes 
0.7769 0.7682 0.7305 0.6633 0.6514

Success - our predicted result was correct! We get 'dog' as the highest predicted result after the one we used (cat). We can think of this scenario as cats say meow and dogs say bark.

## Parent example

Let's move on to the parent example:

In [36]:
# mom - girl + boy should equal dad
dad <- word_vectors["mom", , drop = FALSE] -
  word_vectors["girl", , drop = FALSE] +
  word_vectors["boy", , drop = FALSE]
  
# Calculates pairwise similarities between the rows of two matrices
dad_cos_sim <- sim2(x = word_vectors, y = dad, method = "cosine", norm = "l2")

# Top five predictions
head(round(sort(dad_cos_sim[,1], decreasing = TRUE), digits = 4), 5)

mom    dad   said     my   says 
0.8650 0.7863 0.7067 0.6561 0.6497

'Dad' was a top result. Finally, let's try the infamous king and queen example.



## King and queen example

In [38]:
# king - man + woman should equal queen
queen <- word_vectors["king", , drop = FALSE] -
  word_vectors["man", , drop = FALSE] +
  word_vectors["woman", , drop = FALSE]

# Calculate pairwise similarities
queen_cos_sim = sim2(x = word_vectors, y = queen, method = "cosine", norm = "l2")

# Top five predictions
head(round(sort(queen_cos_sim[,1], decreasing = TRUE), digits = 4), 5)

king   martin careless   luther      lil 
  0.7068   0.6126   0.5958   0.5952   0.5830

Unfortunately, queen came in at 4th. Let's try changing **man** and **woman** to **boy** and **girl** to account for the kid's writting.

In [39]:
# king - boy + girl should equal queen
queen <- word_vectors["king", , drop = FALSE] -
  word_vectors["boy", , drop = FALSE] +
  word_vectors["girl", , drop = FALSE]

# Calculate pairwise similarities
queen_cos_sim = sim2(x = word_vectors, y = queen, method = "cosine", norm = "l2")

# Top five predictions
head(round(sort(queen_cos_sim[,1], decreasing = TRUE), digits = 4), 5)

king  queen   kong     dr   blur 
0.8683 0.6602 0.6104 0.6078 0.5735

It worked!

# A Tangent on Bias

As we discussed in class, word embeddings have proven to be a useful tool for uncovering/revealing bias in large corpora. Here, we can see how well the kids fare. We'll look at occupations. 

In [43]:
job <- word_vectors["job", , drop = FALSE] -
  word_vectors["boy", , drop = FALSE] +
  word_vectors["girl", , drop = FALSE]

# Calculate pairwise similarities
job_cos_sim = sim2(x = word_vectors, y = job, method = "cosine", norm = "l2")

# Top five predictions
head(round(sort(job_cos_sim[,1], decreasing = TRUE), digits = 4), 5)

job    art   site  brain  least 
0.7364 0.5116 0.4817 0.4759 0.4573

In [45]:
job <- word_vectors["job", , drop = FALSE] -
  word_vectors["girl", , drop = FALSE] +
  word_vectors["boy", , drop = FALSE]

# Calculate pairwise similarities
job_cos_sim = sim2(x = word_vectors, y = job, method = "cosine", norm = "l2")

# Top five predictions
head(round(sort(job_cos_sim[,1], decreasing = TRUE), digits = 3), 5)

job   test    bob soccer   wife 
 0.837  0.546  0.506  0.495  0.489

Interesting! We're not seeing the same dynamics observed in other settings. Given the small corpus, though, we'd want to add a lot more data before we could be confident that those biases weren't present here.

# Working with Estimated Embeddings

With that in hand, we can estimate a simple clustering algorithm. We specify 5 clusters, but feel free to play around with that number.

In [89]:
set.seed(12345)
n_means <- 5
clusters <- kmeans(word_vectors, centers = n_means, iter.max  = 30)

Note what we have estimated with KMeans. We have 5 cluster centers, each of 50 dimensions, the same number of dimensions that we have for each of our tokens. Therefore, we look for which of the tokens are most similar to one of our cluster centers. 

In [90]:
cluster1 <- t(as.matrix(clusters$centers[1,]))

In [91]:
clus_cos_sim = sim2(x = word_vectors, y = cluster1, method = "cosine", norm = "l2")


In [92]:
# Top ten cluster words
head(sort(clus_cos_sim[,1], decreasing = TRUE), 10)

gripping    phases     roofs    lowest   kindest freshness  elements    jumpin 
0.6586099 0.6584103 0.6579323 0.6572661 0.6473516 0.6356365 0.6317597 0.6311562 
 swerving    wisper 
0.6287111 0.6242360

Now let's loop over the cluster centers:

In [93]:
topWordMatrix <- matrix(NA, n_means, 10)

for (i in 1:n_means){
  cluster <- t(as.matrix(clusters$centers[i,]))
  clus_cos_sim = sim2(x = word_vectors, y = cluster, method = "cosine", norm = "l2")
  topWordMatrix[i,] <- names(head(sort(clus_cos_sim[,1], decreasing = TRUE), 10))
}

In [94]:
topWordMatrix

gripping,phases,roofs,lowest,kindest,freshness,elements,jumpin,swerving,wisper
lill,noâ,tempted,ore,homesick,backpacks,monroe,sharpening,besties,blunt
werewolf,discrace,tad,platter,doller,rio,blocker,turbo,frankie,louse
but,just,when,because,that,you,now,not,all,if
bag,bathroom,salt,bedroom,cross,thrill,kitchen,bucket,lock,draw


A little hard to see too much coming through in the way of the clusters here. Play around with the specifications to see if shifting the number of clusters in KMeans, the size of the window in GloVe, etc. get us to more sensible clusters. If not, it may just be that the dataset is too limited to really learn much.